### Search Engine With Tools And Agents

In [1]:
## Arxiv--Research
## Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [2]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [3]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [4]:
tools=[wiki,arxiv]

In [5]:
## Custom tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
from dotenv import load_dotenv
load_dotenv()
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F7F2833B30>, search_kwargs={})

In [7]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ")

retriever_tool.name

'langsmith-search'

In [8]:
tools=[wiki,arxiv,retriever_tool]

User Query -> LLM -> (Decision) -> 
    - If needs external data (e.g., Wikipedia, Arxiv, retriever):
        - Agent (Wikipedia/Arxiv/Retriever) -> Tools (Fetch relevant data)
        - Results returned to LLM -> LLM generates the final answer
    - If direct answer possible (LLM has sufficient knowledge):
        - LLM generates answer directly
User Receives Final Answer


In [9]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Quadrant\\anaconda3\\envs\\ai\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001F7DB3B3C40>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F7F2833B3

In [10]:
## Run all this tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()
import os

groq_api_key=os.getenv("GROQ_API_KEY")
openai.api_key=os.getenv("OPENAI_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

In [11]:
## Prompt Template
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
## Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent


RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [13]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [14]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prototyping, to beta testing, to production.

Skip to main conten

{'input': 'Tell me about Langsmith',
 'output': 'Thank you for calling the tool and providing the result for tool call id "call_q2b5". Based on the output, I can provide more information about Langsmith.\n\nLangsmith is a platform for building production-grade LLM (Large Language Model) applications. It allows developers to closely monitor and evaluate their applications, enabling them to ship quickly and with confidence. Langsmith provides observability features, evaluation mechanisms, and prompt engineering tools to help developers build and optimize their AI applications.\n\nSome of the key features of Langsmith include:\n\n1. Observability: Langsmith allows developers to analyze traces, configure metrics, and set up dashboards to monitor their application\'s performance.\n2. Evaluation: Langsmith provides a suite of evaluation tools to help developers test and optimize their applications. This includes creating and running high-quality evaluations, analyzing results, and collecting

In [15]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks wit
Invoking: `wikipedia` with `{'query': 'supervised learning'}`


Page: Supervised learning
Summary: In machine learning, supervised learning (SL) is a paradigm where a model is trained using input objects (e.g. a vector of predictor variables) and desired output values (also known as a supervisory signal), which a
Invoking: `arxiv` with `{'query': 'machine learning supervised learning'}`


Published: 2019-04-05
Title: Is 'Unsupervised Learning' a Misconceived Term?
Authors: Stephen G. Odaibo
Summary: Is all of machine learning supervised to some degree? The field of machine
learning has traditionally been categorized pedagogically into
Invo

{'input': 'What is machine learning',
 'output': "I'll stop using tools now. It seems like the provided tools are not relevant to answering the user's question about machine learning. Instead, I'll provide a direct response.\n\nMachine learning is a subfield of artificial intelligence that involves developing algorithms and statistical models that enable computers to learn from data and make predictions or decisions without being explicitly programmed. It involves training models on large datasets, and the models can then be used to make predictions or classify new data.\n\nMachine learning is a broad field that encompasses many subfields, including supervised learning, unsupervised learning, and reinforcement learning. Supervised learning involves training models on labeled data, where the correct output is provided for each input. Unsupervised learning involves training models on unlabeled data, where the goal is to discover patterns or structure in the data. Reinforcement learning i

In [16]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c

APIError: Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.

In [ ]:
agent_executor.invoke({"input":"what is the python new version?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'what is the latest version of python'}`


Page: Python (programming language)
Summary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.
Python is dynamically type-checked and garbage-colle
Invoking: `wikipedia` with `{'query': 'what is the latest version of python'}`


Page: Python (programming language)
Summary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.
Python is dynamically type-checked and garbage-colleThe latest version of Python is Python 3.10.4.

> Finished chain.


{'input': 'what is the python new version?',
 'output': 'The latest version of Python is Python 3.10.4.'}

In [ ]:
agent_executor.invoke({"input":"What is the latest research on quantum computing?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'latest research on quantum computing'}`


Published: 2023-01-27
Title: A Living Review of Quantum Computing for Plasma Physics
Authors: Óscar Amaro, Diogo Cruz
Summary: Quantum Computing promises accelerated simulation of certain classes of
problems, in particular in plasma physics. Given th
Invoking: `wikipedia` with `{'query': 'quantum computing'}`


Page: Quantum computing
Summary: A quantum computer is a computer that exploits quantum mechanical phenomena. On small scales, physical matter exhibits properties of both particles and waves, and quantum computing leverages this behavior using specia
Invoking: `langsmith-search` with `{'query': 'quantum computing research 2022'}`


Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGr

{'input': 'What is the latest research on quantum computing?', 'output': ''}

In [ ]:
agent_executor.invoke({"input":"what is latest ipl trophy updates in 2025?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'IPL 2025 Trophy'}`


Page: 2025 Indian Premier League
Summary: The 2025 Indian Premier League (also known as IPL 18 and branded as TATA IPL 2025) will be the 18th edition of the Indian Premier League. The tournament will feature ten teams competing in 74 matches from 21 
Invoking: `wikipedia` with `{'query': 'Indian Premier League Trophy'}`


Page: 2025 Indian Premier League
Summary: The 2025 Indian Premier League (also known as IPL 18 and branded as TATA IPL 2025) will be the 18th edition of the Indian Premier League. The tournament will feature ten teams competing in 74 matches from 21 The 2025 Indian Premier League has not yet taken place, so there is no latest trophy update available. However, I can provide you with general information about the IPL trophy. The IPL trophy is officially known as the IPL Trophy and is awarded to the winner of the Indian Premier League. The trophy is designed by the Indian

{'input': 'what is latest ipl trophy updates in 2025?',
 'output': "The 2025 Indian Premier League has not yet taken place, so there is no latest trophy update available. However, I can provide you with general information about the IPL trophy. The IPL trophy is officially known as the IPL Trophy and is awarded to the winner of the Indian Premier League. The trophy is designed by the Indian jewelry designer, Sreejith Sen.\n\nIf you would like to know more about the IPL or its previous winners, I'd be happy to help."}